## 1. データをロードする

In [33]:
input_path="./data/datalake"
output_path="./data/datamart"

In [34]:
from glob import glob
import pandas as pd
csv_files=glob(input_path+"/*.csv")

#csvファイルの中身を追加していくリストを用意
data_list = []

#読み込むファイルのリストを走査
for file in csv_files:
    data_list.append(pd.read_csv(file, encoding="shift-jis"))

#リストを全て行方向に結合
#axis=0:行方向に結合, sort
df = pd.concat(data_list, axis=0, sort=True)

## 2. データの変換処理

### 余計なカラムの削除

In [35]:
df1=df[df["計算対象"]==1].drop(["計算対象","振替"],axis=1)

### カラムを追加・修正する

日付カラムをdatetimeに変更
年月、年のカラムを追加

In [36]:
df2=df1.copy()
df2["日付"]= pd.to_datetime(df2['日付'])
df2["年月"]= pd.to_datetime(df2['日付']).dt.strftime("%Y-%m")
df2["年"]= pd.to_datetime(df2['日付']).dt.strftime("%Y")

- 年間フラグの作成
    - 中項目が1から始まっていたら年間フラグカラムに1をセット
    - 中項目が0から始まっていたら年間フラグに0をセット
    - 中項目が0もしくは1から始まっていなければ、99をセットする

In [37]:
def check_year_flag(string):
    if string.startswith("1"):
        return 1
    else:
        if string.startswith("0"):
            return 0
        else:
            return 99

In [38]:
df3=df2.copy()
df3["年間フラグ"]=df3.apply(lambda row: check_year_flag(row.中項目),axis=1)
df3.head()


,ID,メモ,中項目,保有金融機関,内容,大項目,日付,金額（円）,年月,年,年間フラグ
0,WZcAtFdIgDDMyjgNuSrzCA,フルーツジュース,0交際費_カフェ,LINE Pay,決済 三井アウトレットパーク木更津,交際費,2021-04-30,-496,2021-04,2021,0
1,xmrlnXg358el0fNuBAtNuQ,うどん 交際費,0交際費_外食,LINE Pay,決済 三井アウトレットパーク木更津,交際費,2021-04-30,-520,2021-04,2021,0
2,99tNYHpy_AXAmMSa-ljHkA,うどん,0外食,LINE Pay,決済 三井アウトレットパーク木更津,食費,2021-04-30,-800,2021-04,2021,0
3,sqUWP4TMTTlg4LUfwvwsPQ,NaN,0その他月払い,エポスカード,マネ-フオワ-ド,住宅,2021-04-30,-500,2021-04,2021,0
4,nW9Jbcw8D9ulHgmVEGw3aQ,バック,1自分_年払い,エポスカード,ミツイアウトレツトパ-クキサラヅ,交際費,2021-04-30,-57750,2021-04,2021,1


## 3. データを蓄積する

In [39]:
output_csv_file=output_path+"/output3.csv"

### csvファイルに出力

In [40]:
df3.to_csv(output_csv_file,index=False)

### dbに保存

In [41]:
import pandas as pd
import datetime
from sqlalchemy import create_engine


connection_config = {
    'user': 'metabase',
    'password': 'metabase',
    'host': 'postgres',
    'port': '5432',
    'database': 'metabase'
}

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))
df = pd.read_csv(output_csv_file)
df.to_sql('kakeibo', con=engine, if_exists='replace', index=False)

---

---